In [3]:
import os
import re
import time
from datetime import datetime

import cv2
import numpy as np
import pandas as pd
import pytesseract

from pdf2image import convert_from_path
from PIL import Image

import camelot
import tabula
import pdfplumber

import glob

In [5]:
def read_image(pdf):
    text = tabula.read_pdf(pdf, pages='all')
    return text

In [6]:
filepath = 'C:\\Users\\Lenovo\\Downloads\\ICICI_Bank_Statement_New.pdf'

In [7]:
text = read_image(filepath)
text[0]

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'
Got stderr: Aug 13, 2024 3:29:40 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 13, 2024 3:29:41 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



,Type of Account,Account Number,Balance (INR),MICR,IFSC,Nomination
0,Current,666405112451,"27,314.45 Cr",311229004.0,ICIC0006664,Registered
1,NaN,TOTAL,"27,314.45 Cr",NaN,NaN,NaN


In [4]:
import tabula


print(tabula.__version__)

2.9.3


In [10]:
import site
print(site.getsitepackages())

['c:\\Users\\Lenovo\\anaconda3\\envs\\tesseract_env', 'c:\\Users\\Lenovo\\anaconda3\\envs\\tesseract_env\\lib\\site-packages']


In [159]:
def classify_bank(text):
    '''
    Takes OCR text from first page and detects IFSC code, to infer the bank by using it
    '''

    banks = {"YES": "YES BANK", "ALLA": "ALLAHABAD BANK", "ICIC": "ICICI BANK"}
    ifsc = get_ifsc(text)
    bank = ""
    for j in banks.keys():
        if j in ifsc:
            bank = banks[j]
            break
    
    return bank

#### IFSC #####
def get_ifsc(text):
    
    def replace(text):
        return text.replace('?', '7')
    
    ifsc = text.find('IFSC')
    new_text = text[ifsc : ifsc + 100]
    new_text = replace(new_text)
    # print('new_text---> ', new_text)

    pattern = r'\b[A-Za-z]{1,10}[0-9]{1,10}\b'

    code = re.findall(pattern, new_text)
    
    return [c for c in code if len(c) == 11][0]

In [160]:
# get_ifsc(text)
classify_bank(text)

'ALLAHABAD BANK'

In [ ]:
pdf = pdfplumber.open(file)
#specify pages
page = pdf.pages[page_number]
#extract text from pdf
text = page.extract_text()

In [233]:
filepath = 'C:\\pocs\\bank_statement\\pdfs\\bank1.pdf'
pdf = pdfplumber.open(filepath)
#specify pages
page = pdf.pages[0]
text = page.extract_text()

In [234]:
text

'Allahabad Bank\nLUCKNOW JANAKIPURAM\nADRASH COMPLEX, SECTOR-E , SITAPUR RD, PLAN RING ROAD, LUCKNOW , UTTAR PRADESH\nIFSC Code : ALLA0211993\nAccount Number : 50439602642\nProduct type :  SB\nMr. MOHD AZFAR  ALAM LARI\nC--2/255\nSECTOR  F\nJANKIPURAM           Lucknow  M Corp\nLucknow\nEmail : azfarlari1999@gmail.com\nNOMINATION REGD :  NO\nCleared Balance :853.49\nSTATEMENT OF ACCOUNT from 01/10/2019 to 31/12/2019\n \nPost Date Value Description DR CR Balance\nDate\n02/10/2019 02/10/2019 ATM WDL ~  ~      1000.00 686.27 CR\n+NEW ASHOK\nNAGAR\n00000SECNM867\n005626900000\n04/10/2019 04/10/2019 BY TRANSFER ~  ~       220.00 906.27 CR\nUPI -927721165159-\nshysaksham128@ok\nhdfcbank(SAK\n07/10/2019 07/10/2019 TO TRANSFER ~  ~        84.26 822.01 CR\nUPI -928036903335-\nadd-\nmoney@paytm(Payt\nm) NOTES:Oi\n07/10/2019 07/10/2019 BY TRANSFER ~  ~      5000.00 5822.01 CR\nNEFT-IDFB-HUEY\nTECH PRIVATE -\nIDFBH19280489297\n08/10/2019 08/10/2019 TO TRANSFER ~  ~       100.00 5722.01 CR\nUPI -92

In [184]:
# print(text)
new_text = ' '.join(s for s in text.split('\n'))
new_text
get_ifsc(new_text)

'YESB0000639'

In [38]:
print(new_text)

Page 1 of 2 [IVW_306017_10.83.32.120_20190806145520]  ATULKABRAA@GMAIL.COM 311001 Your Details With Us: M/S.COSMOS ENTERPRISES CO ATUL ELECTRONICS OPP THE BANK OF RAJ GANDHI NAGAR BHILWARAQ & ABKPK5579R BHILWARA RAJASTHAN - INDIA - 311001 Your Base Branch: SUBHASH NAGAR, AJMER ROAD, BHILWARA,BHILWARA,311001 Summary of Account as on 31-07-2019 I. Operative Account in INR Type of Account Account Number Balance (INR) MICR IFSC Nomination Current 666405112451 27,314.45 Cr 311229004 ICIC0006664 Registered TOTAL 27,314.45 Cr Statement of transactions in Current account number: 666405112451 in INR For the period 01-07-2019 To 31-07-2019 Date Particulars Chq.No. Withdrawals Deposits Autosweep Reverse Balance(INR) Sweep 01-07-2019 B/F 16,674.45 Cr 01-07-2019 BY CASH  - BIJOLIA 0.00 2,000.00 18,674.45 Cr 02-07-2019 BIL/INFT/001743804629/NA/ 0.00 3,740.00 22,414.45 Cr 02-07-2019 MMT/IMPS/918317526479/vidrc/MKELECTRON/BANK OF BAR 0.00 3,000.00 25,414.45 Cr 03-07-2019 BY CASH-KACHHOLA 0.00 3,000.00

In [56]:
new_text = '56746653895'
code = re.findall(r'[^A-Z0-9]{11}', new_text)
code

[]

In [227]:
def get_acc(text):
    '''
    From the extracted text, searches for the Account Number
    '''
    
    # if '-' in list(text):
    #     text = text.replace('-', '')
    # print(text)
    index = re.search('account.*n', text.lower()).start()
    # print('index---> ', type(index.start()))
    # print(text[index.start():])
    try:
        text = re.findall(r'[0-9]{9,18}', text[index:])[0]
    except:
        return 0
    return text

In [228]:
# print(type(text))
get_acc(text)


'063991900002710'

In [180]:
def get_name(info):
    '''
    From the array of text, searches for the Account Holder Name
    '''
    # print(text)
    title = ["mr.", "shri", "ms.", "mrs."]
    for i in info.split('\n'):
        # print(i)
        for j in title:
            if j in i.lower():
                # return (i.lower().replace(j, "").upper())
                return i
    return (-1)

In [181]:
get_name(text)

'Mr. MOHD AZFAR  ALAM LARI'

In [145]:
def get_transactions(filepath):
    text = tabula.read_pdf(filepath, pages='all', guess=True)
    return text

t = get_transactions(filepath)

Got stderr: Jul 26, 2024 5:01:08 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jul 26, 2024 5:01:11 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [150]:
j = t[1]

In [151]:
j

,Date,Unnamed: 0,Particulars,Chq.No.,Withdrawals,Deposits,Autosweep,Reverse,Balance(INR)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sweep,NaN
1,01-07-2019,B/F,NaN,NaN,NaN,NaN,NaN,NaN,"16,674.45 Cr"
2,01-07-2019,BY CASH - BIJOLIA,NaN,NaN,0.00,"2,000.00",NaN,NaN,"18,674.45 Cr"
3,02-07-2019,BIL/INFT/001743804629/NA/,NaN,NaN,0.00,"3,740.00",NaN,NaN,"22,414.45 Cr"
4,02-07-2019,MMT/IMPS/918317526479/vidrc/MKELECTRON/BANK OF...,NaN,NaN,0.00,"3,000.00",NaN,NaN,"25,414.45 Cr"
5,03-07-2019,BY CASH-KACHHOLA,NaN,NaN,0.00,"3,000.00",NaN,NaN,"28,414.45 Cr"
6,03-07-2019,MMT/IMPS/918418038560/d2h/HANSHRAJSO/BANK OF,NaN,NaN,0.00,"14,000.00",NaN,NaN,"42,414.45 Cr"
7,NaN,BAROD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,05-07-2019,CLG/COSMOSENTERPRISES /BRK,NaN,5181.0,"40,000.00",0.00,NaN,NaN,"2,414.45 Cr"
9,06-07-2019,MMT/IMPS/918710930334/d2h/HANSHRAJSO/BANK OF,NaN,NaN,0.00,"5,000.00",NaN,NaN,"7,414.45 Cr"


In [156]:
# Read the PDF file, specifying the pages to read
# 'all' specifies to read all pages, or you can specify a list of pages [1, 2, 3, ...]
df_list = tabula.read_pdf(filepath, pages='all', stream=True, area=(320.0, 9.0, 743.81, 601.11), multiple_tables=False)

# Concatenate all DataFrames into one
all_tables = pd.concat(df_list, ignore_index=True)

# Print the combined DataFrame
# print(all_tables)

# Optionally, convert the DataFrame to a dictionary
dict_output = all_tables.to_dict(orient='records')
print(dict_output)

Got stderr: Jul 26, 2024 5:06:28 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



[{'Date': nan, 'Unnamed: 1': nan, 'Particulars': nan, 'Chq.No.': nan, 'Withdrawals': nan, 'Deposits': nan, 'Autosweep': nan, 'Reverse': 'Sweep', 'Balance(INR)': nan}, {'Date': '01-07-2019', 'Unnamed: 1': 'B/F', 'Particulars': nan, 'Chq.No.': nan, 'Withdrawals': nan, 'Deposits': nan, 'Autosweep': nan, 'Reverse': nan, 'Balance(INR)': '16,674.45 Cr'}, {'Date': '01-07-2019', 'Unnamed: 1': 'BY CASH  - BIJOLIA', 'Particulars': nan, 'Chq.No.': nan, 'Withdrawals': '0.00', 'Deposits': '2,000.00', 'Autosweep': nan, 'Reverse': nan, 'Balance(INR)': '18,674.45 Cr'}, {'Date': '02-07-2019', 'Unnamed: 1': 'BIL/INFT/001743804629/NA/', 'Particulars': nan, 'Chq.No.': nan, 'Withdrawals': '0.00', 'Deposits': '3,740.00', 'Autosweep': nan, 'Reverse': nan, 'Balance(INR)': '22,414.45 Cr'}, {'Date': '02-07-2019', 'Unnamed: 1': 'MMT/IMPS/918317526479/vidrc/MKELECTRON/BANK OF BAR', 'Particulars': nan, 'Chq.No.': nan, 'Withdrawals': '0.00', 'Deposits': '3,000.00', 'Autosweep': nan, 'Reverse': nan, 'Balance(INR)': 

In [260]:
pdf_path = 'C:\\pocs\\bank_statement\\pdfs\\bank1.pdf'
d = camelot.read_pdf(pdf_path, pages='all')

df = pd.DataFrame(columns=d[0].df.columns)
for i in range(len(d)):
    df = pd.concat([df, d[i].df], axis=0)

df = df.reset_index()
df = df.drop(["index"], axis=1)

df.columns = ['Post Date', 'Value Date', 'Description', 'Debit', 'Credit', 'Balance']

for i in df.index:
    try:
        l = df["Value Date"][i].split()
        df["Post Date"][i] = l[0]
        df["Value Date"][i] = l[1]
        p = " ".join(l[2:]) + df["Description"][i]
        df["Description"][i] = p
    except:
        continue
df = df.drop(0, axis=0)
df


In [438]:
##### for icici bank #########

pdf_path = 'C:\\pocs\\bank_statement\\pdfs\\yes1.pdf'

t = tabula.read_pdf(pdf_path, pages='all')
d = camelot.read_pdf(pdf_path, pages='all')
new_df = pd.DataFrame(columns=d[0].df.columns)

sflag = 0
eflag = 0

# print(len(t[1].columns))
# print(len(t[2].columns))

# new_df = t[1]
# if "Unnamed: 0" in t[1].columns:
#     t = new_df.drop(["Unnamed: 0"], axis=1)
# if "Particulars" in t[1].columns:
#     new_df = pd.concat([new_df, t[1]], axis=0)

# print(t[1].shape)
# print(t[1])

# print(new_df)


# print(t[2])

# last_row = t[2].iloc[-1:]
# df_shifted = t[2].shift(1)
# df_shifted.iloc[0, :] = df_shifted.columns
# df_shifted = pd.concat([df_shifted, last_row], axis=0)
# df_shifted.columns = new_df.columns
# print(df_shifted)

for i in range(len(t)):
    if 'Date' in t[i].columns:
        new_df = t[i]
        if 'Unnamed: 0' in t[i].columns:
            new_df['Particulars'] = new_df['Unnamed: 0']
            new_df = new_df.drop(['Unnamed: 0'], axis=1)
        sflag = 1
        continue
    if sflag == 1 and eflag == 0:
        last_row = t[i].iloc[-1:]
        df_shifted = t[i].shift(1)
        df_shifted.iloc[0, :] = df_shifted.columns
        df_shifted = pd.concat([df_shifted, last_row], axis=0)
        df_shifted.columns = new_df.columns
        new_df = pd.concat([new_df, df_shifted], axis=0)

        breaking_word = t[i].applymap(lambda x: 'Page Total:' in str(x)).any().any()

        if breaking_word:
            new_df = new_df.reset_index(drop=True)
            print(new_df)
            break
        # print(new_df)
        # break
        # continue
        
    # k2 = list(map(lambda x:str(x).lower(), list(new_df.iloc[-1, :])))
    # print(new_df)
    # print(k2)
    
    # if any('page total' in s for s in k2):
        # new_df = pd.concat([new_df, t[i]], axis=0)
        # break
    # print(k2)

# new_df


# for i in range(len(t)):
#     if 'date' in t[i][0].lower():
#         print('yes')


# d[0].df

# df = pd.DataFrame(columns=d[0].df.columns)

# for i in range(len(d)):
#     df = pd.concat([df, d[i].df], axis=0)


# df.columns = df.iloc[0, :]
# df = df.iloc[1:, :]
# df.columns = [c.replace('\n', ' ') for c in df.columns]
# df


# for i in df.index:
#     try:
#         l = df["Value Date"][i].split()
#         df["Post Date"][i] = l[0]
#         df["Value Date"][i] = l[1]
#         p = " ".join(l[2:]) + df["Description"][i]
#         df["Description"][i] = p
#     except:
#         continue
# df = df.drop(0, axis=0)
# df


Got stderr: Jul 30, 2024 6:56:43 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Jul 30, 2024 6:56:43 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace



IndexError: list index out of range

In [404]:
t[2]

,29-07-2019,UPI/921024499579/Payment from Ph/gopalsoni88@ybl/H,Unnamed: 0,0.00,"2,000.00",Unnamed: 1,Unnamed: 2,"26,314.45 Cr"
0,31-07-2019,MMT/IMPS/921211070844/MB: PCS NAYAK/SHRAWAN\rK...,NaN,0.0,"1,000.00",NaN,NaN,"27,314.45 Cr"
1,Page Total:,"1,15,000.00","1,25,640.00",0.0,0.00,"27,314.45 Cr",NaN,NaN


In [ ]:
import PyPDF2

reader = PyPDF2.PdfFileReader(pdf_path)
text = ""
for page_num in range(reader.numPages):
    page = reader.getPage(page_num)
    text += page.extract_text()

print(text)

In [289]:
df.shape

(38, 8)

In [262]:
df.columns = df.iloc[0, :]
df.columns

Index(['Post Date', 'Value\nDate', 'Description', 'DR', 'CR', 'Balance'], dtype='object', name=0)

In [3]:
import tabula
import pandas as pd

In [6]:
##### for icici bank #########

pdf_path = 'C:\\pocs\\bank_statement\\pdfs\\ICICI_Bank_Statement_New.pdf'

t = tabula.read_pdf(pdf_path, pages='all')
# d = camelot.read_pdf(pdf_path, pages='all', flavor='stream')
# new_df = pd.DataFrame(columns=d[0].df.columns)

# print(d[0].df.columns)

sflag = 0
eflag = 0

# print(t[0])


for i in range(len(t)):
    if 'Date' in t[i].columns or 'Description' in t[i].columns:
        new_df = pd.DataFrame(columns=t[i].columns)
        new_df = t[i]
        if 'Unnamed: 0' in t[i].columns:
            new_df['Particulars'] = new_df['Unnamed: 0']
            new_df = new_df.drop(['Unnamed: 0'], axis=1)
        sflag = 1
        continue
    if sflag == 1 and eflag == 0:
        last_row = t[i].iloc[-1:]
        df_shifted = t[i].shift(1)
        df_shifted.iloc[0, :] = df_shifted.columns
        df_shifted = pd.concat([df_shifted, last_row], axis=0)
        df_shifted.columns = new_df.columns
        new_df = pd.concat([new_df, df_shifted], axis=0)

        breaking_word = t[i].applymap(lambda x: 'Page Total:' in str(x)).any().any()

        if breaking_word:
            new_df = new_df.reset_index(drop=True)
            print(new_df)
            break
   

Got stderr: Aug 01, 2024 4:24:14 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 01, 2024 4:24:15 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



           Date                                        Particulars  \
0           NaN                                                NaN   
1    01-07-2019                                                B/F   
2    01-07-2019                                 BY CASH  - BIJOLIA   
3    02-07-2019                          BIL/INFT/001743804629/NA/   
4    02-07-2019  MMT/IMPS/918317526479/vidrc/MKELECTRON/BANK OF...   
5    03-07-2019                                   BY CASH-KACHHOLA   
6    03-07-2019       MMT/IMPS/918418038560/d2h/HANSHRAJSO/BANK OF   
7           NaN                                              BAROD   
8    05-07-2019                        CLG/COSMOSENTERPRISES  /BRK   
9    06-07-2019       MMT/IMPS/918710930334/d2h/HANSHRAJSO/BANK OF   
10          NaN                                              BAROD   
11   08-07-2019       MMT/IMPS/918814106619/d2h/HANSHRAJSO/BANK OF   
12          NaN                                              BAROD   
13   08-07-2019     

In [144]:
##### for yes bank #########

pdf_path = 'C:\\pocs\\bank_statement\\pdfs\\ICICI_Bank_Statement_New.pdf'

t = tabula.read_pdf(pdf_path, pages='all')
# d = camelot.read_pdf(pdf_path, pages='all', flavor='stream')
new_df = pd.DataFrame()

# print(d[0].df.columns)

sflag = 0
eflag = 0


# Words or phrases to check
breakwords = ['pagetotal', 'closingbalance']

def traverse_cell(df):
    cells = []
    for index, row in df.iterrows():
        for col in df.columns:
            cell_value = row[col]
            cells.append(cell_value)
    return cells


for i in range(len(t)):

    if not t[i].empty:

        # print('t[{}]'.format(i))
        # print(t[i])
        # print('---------------------------------------------------')

        cells = traverse_cell(t[i])
        final_words = []
        final_words.extend(t[i].columns)
        final_words.extend(cells)
        # print(final_words)

        for a in final_words:
            a_clean = str(a).lower().replace(' ', '')  # Lowercase and remove spaces once
            if any(b in a_clean for b in breakwords):
                eflag = 1
                break
        
        if eflag:
            break
        
        transaction_cols = [col for col in t[i].columns if 'Transaction' in col or 'Value' in col or 'Reverse' in col]
        if transaction_cols:
            if 'transaction' in transaction_cols[0].lower():
                t[i] = t[i].rename(columns={transaction_cols[0]: 'Transaction Date'})
            elif 'value' in transaction_cols[0].lower():
                t[i] = t[i].rename(columns={transaction_cols[0]: 'Value Date'})
            elif 'reverse' in transaction_cols[0].lower():
                t[i] = t[i].rename(columns={transaction_cols[0]: 'Reverse Sweep'})
            t[i].drop(0, axis=0, inplace=True)
    
        if ('Date' in t[i].columns or 'Description' in t[i].columns) and sflag == 0:
            new_df = pd.DataFrame(columns=t[i].columns)
            new_df = t[i]
            if 'Unnamed: 0' in t[i].columns:
                new_df['Particulars'] = new_df['Unnamed: 0']
                new_df = new_df.drop(['Unnamed: 0'], axis=1)
            sflag = 1
            continue
       
        if sflag == 1 and eflag == 0:
        
            if 'Description' not in t[i].columns and 'Withdrawals' not in t[i].columns:
                last_row = t[i].iloc[-1:]
                # print('last row {} -->'.format(i), last_row)
                df_shifted = t[i].shift(1)
                df_shifted.iloc[0, :] = df_shifted.columns
                df_shifted = pd.concat([df_shifted, last_row], axis=0)
                df_shifted.columns = new_df.columns
                # print(df_shifted)
                new_df = pd.concat([new_df, df_shifted], axis=0)

            else:
                new_df = pd.concat([new_df, t[i]], axis=0)
            
            
if 'Unnamed: 0' in new_df.columns:
    new_df.drop(columns='Unnamed: 0', axis=1, inplace=True)
new_df = new_df.reset_index(drop=True)
# new_df.to_excel('sample3.xlsx')
new_df

        #     # breaking_word = t[i].applymap(lambda x: 'Page Total:' in str(x)).any().any()
        
   

Got stderr: Aug 05, 2024 4:31:13 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 05, 2024 4:31:14 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



,Date,Particulars,Chq.No.,Withdrawals,Deposits,Autosweep,Reverse Sweep,Balance(INR)
0,01-07-2019,B/F,NaN,NaN,NaN,NaN,NaN,"16,674.45 Cr"
1,01-07-2019,BY CASH - BIJOLIA,NaN,0.00,"2,000.00",NaN,NaN,"18,674.45 Cr"
2,02-07-2019,BIL/INFT/001743804629/NA/,NaN,0.00,"3,740.00",NaN,NaN,"22,414.45 Cr"
3,02-07-2019,MMT/IMPS/918317526479/vidrc/MKELECTRON/BANK OF...,NaN,0.00,"3,000.00",NaN,NaN,"25,414.45 Cr"
4,03-07-2019,BY CASH-KACHHOLA,NaN,0.00,"3,000.00",NaN,NaN,"28,414.45 Cr"
5,03-07-2019,MMT/IMPS/918418038560/d2h/HANSHRAJSO/BANK OF,NaN,0.00,"14,000.00",NaN,NaN,"42,414.45 Cr"
6,NaN,BAROD,NaN,NaN,NaN,NaN,NaN,NaN
7,05-07-2019,CLG/COSMOSENTERPRISES /BRK,5181.0,"40,000.00",0.00,NaN,NaN,"2,414.45 Cr"
8,06-07-2019,MMT/IMPS/918710930334/d2h/HANSHRAJSO/BANK OF,NaN,0.00,"5,000.00",NaN,NaN,"7,414.45 Cr"
9,NaN,BAROD,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
arr = ['apples', 'cat', 'boy']

print(['apple' in a for a in arr])

[True, False, False]


In [69]:
arr = ['Opening Balance: 50899.46Total Debits: 112Total Credits: 41Closing Balance: 19.27']
breakwords = ['closingbalance', 'pagetotal']

for a in arr:
    for b in breakwords:
        if b in a.lower().replace(' ', ''):
            print('yes')
            break

yes


In [100]:
a = pd.DataFrame({'a': [1,2,3],
     'b': [3,6,8]})

arr = []

def ok(cell):
    arr.append(cell)
    return arr

k = a.applymap(ok)

                    a                   b
0  [1, 2, 3, 3, 6, 8]  [1, 2, 3, 3, 6, 8]
1  [1, 2, 3, 3, 6, 8]  [1, 2, 3, 3, 6, 8]
2  [1, 2, 3, 3, 6, 8]  [1, 2, 3, 3, 6, 8]


In [106]:
print(9.0 in ['54'])

False
